In [48]:
%load_ext autoreload
%autoreload 2
import sys
from ridge_utils.DataSequence import DataSequence
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
import sasc.viz
import joblib
import dvu
import sys
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR
from neuro.config import repo_dir, PROCESSED_DIR
from neuro import analyze_helper, viz
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
flatmaps_per_question = __import__('06_flatmaps_per_question')
from neurosynth import term_dict, term_dict_rev
import viz
from load_coef_flatmaps import _load_coefs_individual, _load_coefs_full, \
_load_coefs_individual_wordrate, _load_coefs_wordrate, _load_coefs_shapley

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/aug14_neurosynth_gemv'
rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)

100%|██████████| 527/527 [00:57<00:00,  9.17it/s]
/home/chansingh/imodelsx/imodelsx/process_results.py:92: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[k] = df[k].fillna(np.nan)


experiment varied these params: ['subject', 'feature_selection_alpha', 'qa_questions_version', 'use_random_subset_features', 'single_question_idx', 'seed']


In [55]:
# results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/jun16_gpt4'
# rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)

In [ ]:
subject = 'S03'
df_w_shapley35 = _load_coefs_individual(rr,
                                        subject=subject, qa_questions_version='v3_boostexamples_merged')

df_w_shapley_neurosynth = _load_coefs_individual(rr,
                                                 subject, qa_questions_version='v1neurosynth')

In [58]:
# subject = 'S02'
for subject in ['S01', 'S02', 'S03']:
    # df = _load_coefs_individual(
    # rr, subject=subject, qa_questions_version='v1neurosynth')
    # joblib.dump(df, join(PROCESSED_DIR, subject, 'individual_neurosynth.pkl'))

    df = _load_coefs_individual(
        rr, subject=subject, qa_questions_version='v3_boostexamples_merged')
    joblib.dump(df, join(PROCESSED_DIR, subject, 'individual_35.pkl'))

    df = _load_coefs_full(
        rr, subject=subject, qa_questions_version='v3_boostexamples_merged')
    joblib.dump(df, join(PROCESSED_DIR, subject, 'full_35.pkl'))

    df = _load_coefs_full(
        rr, subject=subject, qa_questions_version='v1neurosynth')
    joblib.dump(df, join(PROCESSED_DIR, subject, 'full_neurosynth.pkl'))

    df = _load_coefs_shapley(
        rr, subject, qa_questions_version='v3_boostexamples_merged')
    joblib.dump(df, join(
        PROCESSED_DIR, subject, 'shapley_35.pkl'))

    df = _load_coefs_shapley(
        rr, subject, qa_questions_version='v1neurosynth')
    joblib.dump(df, join(PROCESSED_DIR,
                subject, 'shapley_neurosynth.pkl'))

    ########### use old models ###################
    # jointly fitted 35-question model
    # df_w_selected35 = _load_coefs_35questions(subject=subject)

    # individually fitted question models
    # df_w_individual = _load_coefs_individual(rr_shapley, subject=subject)
    # joblib.dump(df_w_individual, join(PROCESSED_DIR,
    # subject, 'individual.pkl'))

    # individually fitted question models *with wordrate
    # df_w_individual_wordrate = _load_coefs_individual_wordrate(
    # subject=subject)

    # wordrate
    # df_w_wordrate_alone = _load_coefs_wordrate(subject=subject)

    # # collate individual dfs #########################
    # # average weights for df_w_selected35 and df_w_individual
    # if subject == 'S02':
    #     df_avg = df_w_selected35.merge(df_w_individual, on='question')
    #     df_avg['weights'] = df_avg.apply(
    #         lambda x: np.mean([x['weights_x'], x['weights_y']], axis=0), axis=1)

    # df_avg_individual = df_w_individual.merge(
    #     df_w_individual_wordrate, on='question')
    # df_avg_individual['weights'] = df_avg_individual.apply(
    #     lambda x: np.mean([x['weights_x'], x['weights_y']], axis=0), axis=1)

    # df_qa_dict = {
    #     'selected35': df_w_selected35,
    #     'individual': df_w_individual,
    #     'individual_wordrate': df_w_individual_wordrate,
    #     'wordrate_alone': df_w_wordrate_alone,
    #     # 'avg': df_avg,
    #     'shapley_neurosynth': df_w_shapley_neurosynth,
    #     'shapley35': df_w_shapley35,
    #     'avg_individual': df_avg_individual
    # }
    # joblib.dump(df_qa_dict, f'df_qa_dict_{subject}.pkl')

100%|██████████| 35/35 [00:08<00:00,  4.00it/s]
